In [1]:
import wrds
import pandas as pd


In [ ]:

# ---------------------------------
# 1) Connect and set date range
# ---------------------------------
db = wrds.Connection()
start, end = "2003-01-01", "2025-09-24"


In [ ]:

# ---------------------------------
# 2) Detect column names (schema differences)
# ---------------------------------
cols = db.get_table('crsp', 'dsf', obs=0).columns

# Exchange code column: hexcd (historical) or exchcd
ex_col = 'hexcd' if 'hexcd' in cols else 'exchcd'

# Share code column: shrcd or hshrcd (if available)
sh_col = 'shrcd' if 'shrcd' in cols else ('hshrcd' if 'hshrcd' in cols else None)

# Only keep common stocks if share code column exists
where_sh = f"AND {sh_col} IN (10, 11)" if sh_col else ""

# ---------------------------------
# 3) SQL query: daily share volume + dollar volume
# ---------------------------------
sql = f"""
    SELECT date,
           CASE WHEN {ex_col} = 1 THEN 'NYSE'
                WHEN {ex_col} = 3 THEN 'NASDAQ' END AS exchange,
           SUM(vol) AS sh_volume,
           SUM(ABS(prc) * vol) AS dollar_volume
    FROM crsp.dsf
    WHERE date BETWEEN '{start}' AND '{end}'
      AND {ex_col} IN (1, 3)   -- 1 = NYSE, 3 = NASDAQ
      {where_sh}               -- optional filter for common stock
      AND vol >= 0
      AND prc IS NOT NULL
      AND prc != 0
    GROUP BY date, exchange
    ORDER BY date, exchange;
"""

# ---------------------------------
# 4) Run query and load into pandas
# ---------------------------------
vol_df = db.raw_sql(sql, date_cols=['date'])

print(vol_df.head())


In [ ]:
daily_totals = (
    vol_df
    .groupby('date', as_index=False)
    .agg(
        total_sh_volume=('sh_volume', 'sum'),
        total_dollar_volume=('dollar_volume', 'sum')
    )
)

print(daily_totals.head())

In [ ]:
daily_totals.to_csv("crsp_daily_volume_all.csv", index=False)